In [48]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

In [49]:
ds = ws.datasets['Hz-CTU13_6.csv']
datat = ds.to_dataframe()

ds = ws.datasets['Hz-CTU13_1.csv']
data = ds.to_dataframe()

In [51]:
print(data.shape)
print(datat.shape)

(12955, 204)
(1492, 204)


In [52]:
def ip_to_num(ip):
    rs = 0
    ip = ip.split('.')
    rs += int(ip[0]) << 24
    rs += int(ip[1]) << 16
    rs += int(ip[2]) << 8
    rs += int(ip[3])
    return rs

In [53]:
data[['src', 'dst']] = data[['src', 'dst']].applymap(ip_to_num)

In [54]:
datat[['src', 'dst']] = datat[['src', 'dst']].applymap(ip_to_num)

In [98]:
datat[['src', 'dst']].head()

,src,dst
0,2468369573,1540655006
1,2468369573,2468368393
2,2468369573,2468368393
3,2468369573,1540655006
4,2468369573,1146766602


In [55]:
lst = list(data['protocol'].unique())
dct = {}

for i, item in enumerate(lst):
    print(i, item)
    dct[item] = i
    
data.replace({'protocol': dct}, inplace=True)

0 UDP
1 TCP
2 254
3 ICMP


In [56]:
lst = list(datat['protocol'].unique())
dct = {}

for i, item in enumerate(lst):
    print(i, item)
    dct[item] = i
    
datat.replace({'protocol': dct}, inplace=True)

0 TCP
1 UDP
2 ICMP


In [57]:
data[['src', 'dst', 'protocol']].head()

,src,dst,protocol
0,2468369573,2468369663,0
1,2468369573,2468368393,0
2,2468369573,1249765571,1
3,2468369573,2468369663,0
4,2468369573,4026531834,0


In [ ]:
data['original_label'] = data['original_label'].str.split("-").str[1]
data.loc[datat['original_label'] != 'Botnet', 'original_label'] = 0
data.loc[datat['original_label'] == 'Botnet', 'original_label'] = 1
data['original_label'] = data['original_label'].astype(int)

In [59]:
datat['original_label'] = datat['original_label'].str.split("-").str[1]
datat.loc[datat['original_label'] != 'Botnet', 'original_label'] = 0
datat.loc[datat['original_label'] == 'Botnet', 'original_label'] = 1
datat['original_label'] = datat['original_label'].astype(int)

In [60]:
print(data['original_label'].unique())

[0 1]


In [61]:
data.drop(['original_dataset', 'event_generator', 'event_signature', 'event_priority'], axis=1, inplace=True)

In [62]:
datat.drop(['original_dataset', 'event_generator', 'event_signature', 'event_priority'], axis=1, inplace=True)

In [63]:
res = data['original_label']
data.drop('original_label', inplace=True, axis=1)

In [64]:
rest = datat['original_label']
datat.drop('original_label', inplace=True, axis=1)

In [65]:
print(data.shape, res.shape)
print(datat.shape, rest.shape)

(12955, 199) (12955,)
(1492, 199) (1492,)


In [66]:
#feature extraction

data = data[['protocol', 'src', 'src_port', 'dst', 'dst_port', 'packets', 'bytes', 'flow_duration', 'inter_time_avg',
             'response_rel_time', 'C_duration_avg', 'C_number_of_contacts', 'dst2src_packets', 'src2dst_packets_rate']]

In [67]:
datat = datat[['protocol', 'src', 'src_port', 'dst', 'dst_port', 'packets', 'bytes', 'flow_duration', 'inter_time_avg',
             'response_rel_time', 'C_duration_avg', 'C_number_of_contacts', 'dst2src_packets', 'src2dst_packets_rate']]

In [68]:
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object'] 
numerical_columns   = [c for c in data.columns if data[c].dtype.name != 'object'] 

In [69]:
categorical_columnst = [c for c in datat.columns if datat[c].dtype.name == 'object'] 
numerical_columnst   = [c for c in datat.columns if datat[c].dtype.name != 'object']

In [70]:
# for c in categorical_columns:
#     print (c, data[c].unique())

In [71]:
# data.count(axis=0)

In [72]:
# data_describe = data.describe(include=[object])

In [73]:
# binary_columns = [c for c in categorical_columns if data_describe[c]['unique'] == 2]
# nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2]
# print (binary_columns, nonbinary_columns)

In [74]:
# for c in binary_columns: #Ð½Ð°Ð¹ÑÐ°ÑÑÑÑÑ Ð·Ð½Ð°ÑÐµÐ½Ð½Ñ Ð·Ð°Ð¼ÑÐ½ÑÑÐ¼Ð¾ Ð½Ð° 0
#     top = data_describe[c]['top']
#     data.at[data[c] == top, c] = 0
#     data.at[data[c] != 0, c] = 1

In [75]:
# data_nonbinary = pd.get_dummies(data[nonbinary_columns], dummy_na=False)
# data_nonbinary.astype('int', copy=False)
# # data_nonbinary.dtypes
# # data_nonbinary

In [99]:
data_numerical = data[numerical_columns]
data_numerical = (data_numerical - data_numerical.mean()) / data_numerical.std()
# data_numerical.describe()

In [100]:
data_numericalt = datat[numerical_columns]
data_numericalt = (data_numericalt - data_numericalt.mean()) / data_numericalt.std()
# data_numericalt.describe()

In [101]:
datax = data_numerical

In [102]:
dataxt = data_numericalt

In [266]:
# datax = pd.concat((data_numerical, data[binary_columns], data_nonbinary), axis=1)
# datax = pd.DataFrame(datax, dtype=float)
# print (datax.shape)

(1492, 586)


In [26]:
datax.isnull().values.any()

False

In [248]:
# datax = data.copy()

In [249]:
# datax = datax[['protocol']]

In [130]:
# datax = datax[['protocol', 'src', 'src_port', 'dst', 'dst_port', 'packets', 'bytes', 'flow_duration', 'inter_time_avg',
#              'response_rel_time', 'C_duration_avg', 'C_number_of_contacts', 'dst2src_packets', 'src2dst_packets_rate']]

In [149]:
# datax['protocol'] = datax['protocol'].astype('category')
# datax['src'] = datax['src'].astype('category')
# datax['dst'] = datax['dst'].astype('category')

In [104]:
datax.dtypes

protocol                float64
src                     float64
src_port                float64
dst                     float64
dst_port                float64
packets                 float64
bytes                   float64
flow_duration           float64
inter_time_avg          float64
response_rel_time       float64
C_duration_avg          float64
C_number_of_contacts    float64
dst2src_packets         float64
src2dst_packets_rate    float64
dtype: object

In [83]:
X_train = datax 
X_test = dataxt
y_train = res
y_test = rest

In [103]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(12955, 14) (12955,)
(1492, 14) (1492,)


In [91]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

# y_train_predict = knn.predict(X_train)
# y_test_predict = knn.predict(X_test)

# err_train = np.mean(y_train != y_train_predict)
# err_test  = np.mean(y_test  != y_test_predict)
# print (err_train, err_test)
print(classification_report(y_train, knn.predict(X_train)))
print(classification_report(y_test, knn.predict(X_test)))

             precision    recall  f1-score   support

          0       0.99      0.97      0.98       349
          1       1.00      1.00      1.00     12606

avg / total       1.00      1.00      1.00     12955

             precision    recall  f1-score   support

          0       0.48      0.24      0.32        95
          1       0.95      0.98      0.97      1397

avg / total       0.92      0.93      0.92      1492



In [92]:
from sklearn.svm import SVC #SVM!
svc = SVC()
svc.fit(X_train, y_train)

# err_train = np.mean(y_train != svc.predict(X_train))
# err_test  = np.mean(y_test  != svc.predict(X_test))
# print (err_train, err_test)

print(classification_report(y_train, svc.predict(X_train)))
print(classification_report(y_test, svc.predict(X_test))) #

             precision    recall  f1-score   support

          0       0.99      0.60      0.75       349
          1       0.99      1.00      0.99     12606

avg / total       0.99      0.99      0.99     12955

             precision    recall  f1-score   support

          0       0.39      0.15      0.21        95
          1       0.94      0.98      0.96      1397

avg / total       0.91      0.93      0.92      1492



In [105]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

print(classification_report(y_train, clf.predict(X_train)))
print(classification_report(y_test, clf.predict(X_test))) 

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       349
          1       1.00      1.00      1.00     12606

avg / total       1.00      1.00      1.00     12955

             precision    recall  f1-score   support

          0       0.09      0.64      0.15        95
          1       0.96      0.54      0.69      1397

avg / total       0.90      0.55      0.66      1492



In [93]:
from sklearn import ensemble

rf = ensemble.RandomForestClassifier(n_estimators=82, random_state=10)
rf.fit(X_train, y_train)

# err_train = np.mean(y_train != rf.predict(X_train))
# err_test  = np.mean(y_test  != rf.predict(X_test))
# print (err_train, err_test)

print(classification_report(y_train, rf.predict(X_train)))
print(classification_report(y_test, rf.predict(X_test))) #

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       349
          1       1.00      1.00      1.00     12606

avg / total       1.00      1.00      1.00     12955

             precision    recall  f1-score   support

          0       0.05      0.64      0.09        95
          1       0.84      0.13      0.22      1397

avg / total       0.79      0.16      0.21      1492



In [94]:
#GRADIENTBOOSTING
gbt = ensemble.GradientBoostingClassifier(n_estimators=105, random_state=None)
gbt.fit(X_train, y_train)

# err_train = np.mean(y_train != gbt.predict(X_train))
# err_test = np.mean(y_test != gbt.predict(X_test))
# print (err_train, err_test)

print(classification_report(y_train, gbt.predict(X_train)))
print(classification_report(y_test, gbt.predict(X_test))) #

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       349
          1       1.00      1.00      1.00     12606

avg / total       1.00      1.00      1.00     12955

             precision    recall  f1-score   support

          0       0.09      0.63      0.15        95
          1       0.96      0.54      0.69      1397

avg / total       0.90      0.55      0.66      1492



In [95]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, 
                    hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)

# err_train = np.mean(y_train != clf.predict(X_train))
# err_test = np.mean(y_test != clf.predict(X_test))
# print (err_train, err_test)

print(classification_report(y_train, clf.predict(X_train)))
print(classification_report(y_test, clf.predict(X_test))) #

             precision    recall  f1-score   support

          0       0.93      1.00      0.96       349
          1       1.00      1.00      1.00     12606

avg / total       1.00      1.00      1.00     12955

             precision    recall  f1-score   support

          0       0.44      0.34      0.38        95
          1       0.96      0.97      0.96      1397

avg / total       0.92      0.93      0.93      1492



In [96]:
# Stochastic Gradient Descent

from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="hinge", penalty="l1")
#clf = SGDClassifier(loss="log")
clf.fit(X_train, y_train)

# err_train = np.mean(y_train != clf.predict(X_train))
# err_test = np.mean(y_test != clf.predict(X_test))
# print (err_train, err_test)

print(classification_report(y_train, clf.predict(X_train)))
print(classification_report(y_test, clf.predict(X_test))) #

             precision    recall  f1-score   support

          0       0.92      0.26      0.40       349
          1       0.98      1.00      0.99     12606

avg / total       0.98      0.98      0.97     12955

             precision    recall  f1-score   support

          0       0.05      0.61      0.09        95
          1       0.85      0.15      0.26      1397

avg / total       0.80      0.18      0.25      1492



In [97]:
#Gaussian Naive Bayes

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# err_train = np.mean(y_train != gnb.predict(X_train))
# err_test = np.mean(y_test != gnb.predict(X_test))
# print (err_train, err_test)

print(classification_report(y_train, gnb.predict(X_train)))
print(classification_report(y_test, gnb.predict(X_test))) #

             precision    recall  f1-score   support

          0       0.78      0.58      0.66       349
          1       0.99      1.00      0.99     12606

avg / total       0.98      0.98      0.98     12955

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        95
          1       0.94      1.00      0.97      1397

avg / total       0.88      0.94      0.91      1492



/home/nbuser/anaconda3_23/lib/python3.4/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
